In [ ]:
import subprocess
import os
script_folder='/home/dengw1/workspace/snakerun/MeCP2_iCLIP/scripts'
data_folder='/home/dengw1/workspace/MeCP2_iCLIP'
genome_folder='/home/dengw1/workspace/genome'
sub_folder='{script_folder}/qsub'.format(script_folder=script_folder)
if not os.path.isdir(sub_folder):
    os.makedirs(sub_folder)

In [ ]:
# Demultiplex
dem='python {script_folder}/demultiplex_read1.py {data_folder}/raw_fq/Undetermined_S0_L003_R1_001.fastq {data_folder}/raw_fq'.format(script_folder=script_folder,data_folder=data_folder)
print(subprocess.check_output(dem,shell=True))

# Adapter triming, QC, STAR mapping & rRNA/tRNA masking
cmd='''mkdir -p {data_folder}/cutadapt
cutadapt -a AGATCGGAAGAGCGGTTCAG --quality-cutoff 6 -m 15 -o {data_folder}/cutadapt/{group}.fq {data_folder}/raw_fq/{group}.fq
fastqc -o {data_folder}/cutadapt/ {data_folder}/cutadapt/{group}.fq
mkdir -p {data_folder}/star/{group}/
STAR --genomeDir {genome_folder}/star/{genome} \
--readFilesIn {data_folder}/cutadapt/{group}.fq  --outSAMtype BAM Unsorted \
--outFileNamePrefix {data_folder}/star/{group}/ \
--outFilterMultimapNmax 100 \
--runThreadN 20 \
--alignEndsProtrude 15 ConcordantPair \
--twopassMode Basic --limitOutSJcollapsed 2000000 \
--outStd Log > {data_folder}/star/{group}/log.txt 2>&1
bedtools intersect -f 0.90 -abam {data_folder}/star/{group}/Aligned.out.bam -b {genome_folder}/raw/{genome}/rRNA_tRNA.bed -v > {data_folder}/star/{group}/Aligned.out.mask_rRNA.bam
python {script_folder}/mapping_stat.py {data_folder}/star/{group}/Aligned.out.bam {data_folder}/cutadapt/{group}_fastqc.zip > {data_folder}/star/{group}/mapping_stat.txt
python {script_folder}/collapse_dup.py -b {data_folder}/star/{group}/Aligned.out.mask_rRNA.bam -o {data_folder}/star/{group}/Aligned.out.mask_rRNA.collapsed_dup.bam -m {data_folder}/star/{group}/dup_removal.txt
mkdir -p {data_folder}/clam/{group}
CLAM preprocessor -i {data_folder}/star/{group}/Aligned.out.mask_rRNA.collapsed_dup.bam -o {data_folder}/clam/{group} --read-tagger-method start --lib-type sense
'''
for group in ['M1','M2','M3','H1','H2','H3']:
    genome='hg19'
    if group.startswith('M'):
        genome='mm10'
    cmd_f='%s/qsub/%s.sh'%(script_folder,group)
    with open(cmd_f,'w') as cmd_file:
        cmd_file.write(cmd.format(group=group,genome=genome,data_folder=data_folder,genome_folder=genome_folder,script_folder=script_folder))
    print(subprocess.check_output('qsub -cwd -l h_vmem=40G,m_mem_free=40G %s'%cmd_f,shell=True))
    

In [ ]:
cmd='''genome={genome}

mkdir -p {data_folder}/clam/{group}_{binw}
mkdir -p {data_folder}/clam/peaks_{binw}/{group}
mkdir -p {data_folder}/homer/{binw}/{group}

CLAM realigner -i {data_folder}/clam/{group}/multi.sorted.bam -o {data_folder}/clam/{group}_{binw} --winsize {binw} --max-tags -1 --read-tagger-method start --lib-type sense

echo 'realigned'
CLAM peakcaller -i {data_folder}/clam/{group}/unique.sorted.bam {data_folder}/clam/{group}_{binw}/realigned.sorted.bam \
-o {data_folder}/clam/peaks_{binw}/{group}  --binsize {binw} --qval-cutoff 1 \
--gtf {genome_folder}/raw/{genome}/{genome}.gtf -p 20

python {script_folder}/make_bw.py --ub {data_folder}/clam/{group}/unique.sorted.bam --rb \
    {data_folder}/clam/{group}_{binw}/realigned.sorted.bam -g {genome_folder}/raw/hg19/hg19.gtf -u -t 20 -o {data_folder}/bigwig/{binw} -s {group} -c {genome_folder}/raw/{genome}/chr_size.txt

CLAM permutation_callpeak -i {data_folder}/clam/{group}/unique.sorted.bam {data_folder}/clam/{group}_{binw}/realigned.sorted.bam \
-o {data_folder}/clam/peaks_{binw}/{group} -p 30 \
--gtf {genome_folder}/raw/{genome}/{genome}.gtf --qval-cutoff 1 --merge-size {binw}

mv {data_folder}/clam/peaks_{binw}/{group}/narrow_peak.permutation.bed {data_folder}/clam/peaks_{binw}/{group}/narrow_peak.permutation.bed.all
awk '{{split($4,a,":");if(a[3]<=0.005){{print}}}}' {data_folder}/clam/peaks_{binw}/{group}/narrow_peak.permutation.bed.all > {data_folder}/clam/peaks_{binw}/{group}/narrow_peak.permutation.bed

echo 'peak_called'
findMotifsGenome.pl {data_folder}/clam/peaks_{binw}/{group}/narrow_peak.permutation.bed {genome} {data_folder}/homer/{binw}/{group} \
-rna -len 5,6,7 \
-p 20 -size 100 -S 10
echo 'done'
'''
for group in ['M1','M2','M3','H1','H2','H3']:
    genome='hg19'
    if group.startswith('M'):
        genome='mm10'
    for binw in ['100','150','200']:
        cmd_f='%s/qsub/%s_%s_clam.sh'%(script_folder,group,binw)
        with open(cmd_f,'w') as cmd_file:
            cmd_file.write(cmd.format(group=group,genome=genome,binw=binw,data_folder=data_folder,genome_folder=genome_folder,script_folder=script_folder))
        print(group+'_'+binw)
        print(subprocess.check_output('qsub -cwd -l h_vmem=80G,m_mem_free=40G %s'%cmd_f,shell=True))

In [ ]:
from collections import defaultdict
import pandas as pd
import subprocess
summary=defaultdict(lambda:defaultdict(str))
print(data_folder)
for group in ['M1','M2','M3','H1','H2','H3']:
    for line in open('%s/star/%s/Log.final.out'%(data_folder,group)):
        info=line.strip().split('\t')
        if 'Number of input reads' in line:
            summary[group]['input']=info[1]
        elif 'Average mapped length' in line:
            summary[group]['mapped_len']=info[1]
        elif 'Uniquely mapped reads' in line:
            summary[group]['unique_rate']=info[1]
        elif r'% of reads mapped to multiple loci' in line:
            summary[group]['multi_rate']=info[1]
    summary[group]['masked']=subprocess.check_output('samtools view -c -F 260 %s/star/%s/Aligned.out.mask_rRNA.bam'%(data_folder,group),shell=True).decode('utf-8').strip()
    summary[group]['deduped']=subprocess.check_output('samtools view -c -F 260 %s/star/%s/Aligned.out.mask_rRNA.collapsed_dup.bam'%(data_folder,group),shell=True).decode('utf-8').strip()
    summary[group]['realigned']=subprocess.check_output('samtools view -c -F 260 %s/clam/%s_200/realigned.sorted.bam'%(data_folder,group),shell=True).decode('utf-8').strip()
    summary[group]['peak_unique']=subprocess.check_output('samtools view -c -F 260 %s/clam/%s/unique.sorted.bam'%(data_folder,group),shell=True).decode('utf-8').strip()
pd.DataFrame.from_dict(dict(summary)).to_csv('results/summary.txt')